# Naive Bayes (the easy way)

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [4]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/content/drive/MyDrive/Colab Notebooks/MLCourse/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/content/drive/MyDrive/Colab Notebooks/MLCourse/emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [6]:
data.head()

,message,class
/content/drive/MyDrive/Colab Notebooks/MLCourse/emails/spam/00051.fd20658f0e586d1f27f9396401f4981c,Help wanted. We are a 14 year old fortune 500...,spam
/content/drive/MyDrive/Colab Notebooks/MLCourse/emails/spam/00013.d3f0b591a65f116ea5d9d4ad919f83aa,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
/content/drive/MyDrive/Colab Notebooks/MLCourse/emails/spam/00012.381e4f512915109ba1e0853a7a8407b2,"<table width=""600"" border=""20"" align=""center"" ...",spam
/content/drive/MyDrive/Colab Notebooks/MLCourse/emails/spam/00019.bbc97ad616ffd06e93ce0f821ca8c381,Lowest rates available for term life insurance...,spam
/content/drive/MyDrive/Colab Notebooks/MLCourse/emails/spam/00063.2334fb4e465fc61e8406c75918ff72ed,IS YOUR BUSINESS MAKING MONEY!\n\nSet Up To Ac...,spam


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [8]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)
print(counts)
classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

  (0, 27983)	1
  (0, 57714)	1
  (0, 58215)	4
  (0, 11302)	4
  (0, 1319)	1
  (0, 60483)	1
  (0, 40572)	1
  (0, 24925)	1
  (0, 4827)	1
  (0, 17005)	1
  (0, 53213)	3
  (0, 31369)	4
  (0, 26986)	1
  (0, 11640)	1
  (0, 54226)	1
  (0, 46062)	1
  (0, 35377)	3
  (0, 24806)	2
  (0, 30548)	1
  (0, 58522)	1
  (0, 57713)	1
  (0, 53745)	5
  (0, 58962)	1
  (0, 25222)	3
  (0, 28535)	2
  :	:
  (2999, 51329)	1
  (2999, 44126)	1
  (2999, 37652)	1
  (2999, 17954)	1
  (2999, 30712)	1
  (2999, 10981)	1
  (2999, 59145)	2
  (2999, 19192)	1
  (2999, 38382)	1
  (2999, 36480)	1
  (2999, 26008)	1
  (2999, 48876)	1
  (2999, 28870)	1
  (2999, 24370)	1
  (2999, 22004)	2
  (2999, 16199)	1
  (2999, 27529)	1
  (2999, 46789)	1
  (2999, 54262)	1
  (2999, 23068)	1
  (2999, 39003)	1
  (2999, 18432)	1
  (2999, 46338)	1
  (2999, 53079)	1
  (2999, 57656)	2


MultinomialNB()

Let's try it out:

## Activity

In [9]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [10]:
print(classifier.score())

TypeError: ignored